<a href="https://colab.research.google.com/github/maurojp/XOR_with_GA/blob/master/XOR_with_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparamos el Entorno**

In [0]:
try:
  # El comando %tensorflow_version solo existe en Colaboratory. Es un MAGIC COMMAND.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
import numpy as np
import random as rd

In [0]:
# Set de datos para la puerta XOR
x_train = np.array([[0,0],[0,1],[1,0],[1,1]], dtype = int)
y_train = np.array([[0],[1],[1],[0]], dtype = int)

# **Funciones de Activación**

In [0]:
def relu(z):
    result = z
    result[z < 0] = 0
    return result

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-1 * z))

# **Predicción y Evaluación**
En este método implementamos manualmente la red MLP para predecir y poder evaluar el accuracy de un cromosoma.

In [0]:
def predict_n_evaluate(weights_mat, train_inputs, train_outputs):
    predictions = np.zeros(shape=(train_inputs.shape[0]))
    # Loop de todas la entradas
    for sample_id in range(train_inputs.shape[0]):
      # Input
      y = train_inputs[sample_id, :]
      # Capa oculta
      y = np.matmul(y, weights_mat[0])
      # Funcion de activacion
      y = relu(y)
      # Capa de salida
      y = np.matmul(y, weights_mat[1])
      # Funcion de activacion
      y = sigmoid(y)
      predicted_value = np.round(y)
      predictions[sample_id] = predicted_value
    predictions = np.reshape(predictions, (4, 1))
    # Predicciones correctas
    correct_predictions = np.where(predictions == train_outputs)[0].size
    # Calculo de accuracy
    accuracy = (correct_predictions / train_outputs.size) * 100
    return accuracy, predictions

# **Evaluación de la Población**


In [0]:
def evaluate_population(weights_mat, train_inputs, train_outputs):
    accuracy = np.empty(shape=(len(weights_mat)))
    for chromosome in range(len(weights_mat)):
        chromosome_mat = weights_mat[chromosome]
        accuracy[chromosome], _ = predict_n_evaluate(chromosome_mat, train_inputs, train_outputs)
    return accuracy

# **Operaciones para manipular los cromosomas**
Necesitamos que el cromosoma tenga forma de vector unidimensional para operar en los metodos de cruza, mutación, etc. y también lo necesitamos en la forma de dos matrices para representar los pesos de la red MLP a fin de evaluarlo. Para esto definimos dos funciones que cambian la estructura del cromosoma.

In [0]:
# Convierte el cromosoma de matriz a vector para operaciones del algoritmo genetico
def mat_to_vector(mat_pop_weights):
    pop_weights_vector = []
    for sol_id in range(len(mat_pop_weights)):
        curr_vector = []
        for layer_id in range(len(mat_pop_weights[1])):
            vector_weights = np.reshape(mat_pop_weights[sol_id][layer_id], newshape=(mat_pop_weights[sol_id][layer_id].size))
            curr_vector.extend(vector_weights)
        pop_weights_vector.append(curr_vector)
    return pop_weights_vector

# Convierte el cromosoma de vector a matriz para evaluarlo en la red neuronal
def vector_to_mat(vector_pop_weights, mat_pop_weights):
    list_mat_weights = []
    for sol_id in range(len(mat_pop_weights)):
        start = 0
        end = 0
        mat_weights = []
        for layer_id in range(len(mat_pop_weights[1])):
            end = end + mat_pop_weights[sol_id][layer_id].size
            curr_vector = vector_pop_weights[sol_id][start:end]
            mat_layer_weights = np.reshape(curr_vector, newshape=(mat_pop_weights[sol_id][layer_id].shape))
            mat_weights.append(mat_layer_weights)
            start = end
        list_mat_weights.append(mat_weights)
    return list_mat_weights

# **Selección de los mejores *num_parents* cromosomas como padres**

In [0]:
def select_parents(population, evaluation, num_parents):
    parents = []
    for parent_num in range(num_parents):
        max_evaluation_id = np.where(evaluation == np.max(evaluation))
        max_evaluation_id = max_evaluation_id[0][0]
        parents.append(population[max_evaluation_id])
        evaluation[max_evaluation_id] = -99999999999
    return parents

# **Método de Cruza Uniforme Aleatoria**

In [0]:
def uniform_random_crossover(parents, offspring_size):
    offspring = []
    filter_vec_1 = np.random.randint(2, size=(offspring_size[1]))
    filter_vec_2 = filter_vec_1^1 
    for pair in range(offspring_size[0]):
        # Id del primer padre
        parent1_id = pair%len(parents)
        # Id del segundo padre
        parent2_id = (pair+1)%len(parents)
        # Creación de la descendencia
        offspring.append((parents[parent1_id] * filter_vec_1 + parents[parent2_id] * filter_vec_2).tolist())
    return offspring

# **Función de Mutación**

In [0]:
def mutation(offspring, gen_mutation_percent, cro_mutation_percent):
  # Cantidad de genes que mutan en un cromosoma mutante
  num_mutations_gen = np.uint32((gen_mutation_percent*len(offspring[1]))/100)
  # Indices de los genes que mutan en un cromosoma mutante
  mutation_indices_gen = np.array(rd.sample(range(0, len(offspring[1])), num_mutations_gen))
  # Cantidad de cromosomas que mutan en la poblacion
  num_mutations_cro = np.uint32((cro_mutation_percent*len(offspring))/100)
  # Indices de los cromosomas que mutan
  mutation_indices_cro = np.array(rd.sample(range(0, len(offspring)), num_mutations_cro))
  for id_cro in mutation_indices_cro:
    for id_gen in mutation_indices_gen: 
      random_value = rd.uniform(-0.1, 0.1)
      offspring[id_cro][id_gen] = offspring[id_cro][id_gen] + random_value
  return offspring

# **Ya tenemos todas las funciones necesarias, creamos y configuramos la población inicial**

In [0]:
# Tamaño de la población
sol_per_pop = 16
# Cantidad de padres en la seleccion
num_parents = 8
# Cantidad de generaciones
num_generations = 100
# Porcentaje de genes que mutan en un cromosoma mutante
gen_mutation_percent = 30
# Porcentaje de descendientes que mutan
cro_mutation_percent = 50
# Creación de la población inicial
initial_pop_weights = []
for curr_sol in np.arange(0, sol_per_pop):
  # La capa oculta tiene dos neuronas
  HL_neurons = 2
  input_HL_weights = np.random.uniform(-3, 3, size=(x_train.shape[1], HL_neurons))
  # La capa de salida tiene una neurona
  output_neurons = 1
  HL_output_weights = np.random.uniform(-3, 3, size=(HL_neurons, output_neurons))
  initial_pop_weights.append([input_HL_weights, HL_output_weights])
pop_weights_mat = initial_pop_weights
pop_weights_vector = mat_to_vector(pop_weights_mat)
accuracies = np.empty(shape=(num_generations))

# **Entrenamiento con Algoritmo Genético**

In [0]:
for generation in range(num_generations):
  print("Generación : ", generation)

  # ------------------------------------------------------------------------
  # -- 1. EVALUACION DE LA POBLACION                                      --
  # ------------------------------------------------------------------------

  # Creamos la matrices de pesos a partir de los vectores. (cromosomas)
  pop_weights_mat = vector_to_mat(pop_weights_vector, pop_weights_mat)

  # Evaluamos la poblacion de cromosomas.
  evaluation = evaluate_population(pop_weights_mat, x_train, y_train)
  print("1. Evaluacion de Cromosomas")
  print(evaluation)

  # ------------------------------------------------------------------------
  # -- 2. SELECCION DE LOS MEJORES CROMOSOMAS                             --
  # ------------------------------------------------------------------------

  parents = select_parents(pop_weights_vector, evaluation.copy(), num_parents)
  print("2. Padres Seleccionados")
  print(parents)

  # ------------------------------------------------------------------------
  # -- 3. CRUZA DE LOS CROMOSOMAS SELECCIONADOS                           --
  # ------------------------------------------------------------------------

  crossover = uniform_random_crossover(parents, offspring_size=(len(pop_weights_vector)-len(parents), len(pop_weights_vector[0])))
  print("3. Descendencia por Cruza Aleatoria Uniforme")
  print(crossover)

  # ------------------------------------------------------------------------
  # -- 4. MUTACION DE LA DESCENDENCIA                                     --
  # ------------------------------------------------------------------------ 

  final_offspring = mutation(crossover, gen_mutation_percent, cro_mutation_percent)
  print("4. Descendencia afectada por Mutación")
  print(final_offspring)

  # ------------------------------------------------------------------------
  # -- 5. CREACION DE LA NUEVA POBLACION                                  --
  # ------------------------------------------------------------------------

  pop_weights_vector = parents + final_offspring
  print("5. Nueva Generación")
  print(pop_weights_vector)

Generación :  0
1. Evaluacion de Cromosomas
[75. 75. 75. 50. 75. 50. 50. 50. 75. 50. 75. 50. 50. 50. 75. 50.]
2. Padres Seleccionados
[[-1.3502061885949563, -0.420315083990046, -0.9813952655145908, 0.17615104349180033, -1.5357837300149628, 1.3019654206902702], [-0.287910252406153, -1.5637621946152942, -1.4545457208724504, 0.8484806817031814, -2.6826960208957735, 0.8722498350835552], [-1.0032163594590962, -1.63013879185639, -1.2376974576952648, 1.176995534229742, -1.7820278704302213, 1.2509544606997025], [0.1883525401355075, -0.22426395024679335, -0.8419685068861358, 1.4385742900754366, 0.48203499646277237, 2.46758686566812], [2.528765229903266, -1.4980135267021846, 0.3424470985885897, 2.842312727599822, 0.6835814442986967, 1.2247343232088541], [0.452432545029263, -0.2805813706530822, 1.5137151721277595, -0.8373644695909723, 2.969173052373229, -2.34479168688517], [-1.0347822242484699, -1.458044523258789, -2.0646000848954573, 0.3401384425196188, -0.1725690236601034, 0.7042688576646654], 